
## Racismo

En esta notebook analizamos un poco los resultados sobre la categoría RACISM. Acá Mixtral y ChatGPT tienen una precisión mucho menor. Ver [Resultados](Results_full.ipynb)


In [5]:

%load_ext autoreload
%autoreload 2
import pandas as pd
from glob import glob
from rioplatense_hs.mixtral import post_process_output
from rioplatense_hs.preprocessing import labels as all_labels
pd.options.display.max_colwidth = 1000
#df_train.loc[df_train["LGBTI"] > 0, ["title", "text", "text_label"]].sample(20)

labels = ["WOMEN", "LGBTI", "RACISM", "CLASS"]


files = {
    #"one-shot": sorted(glob("../data/test*pred*1shot.csv")),
    "gpt-3.5": sorted(glob("../data/runs/test_*_chatgpt.csv")),
    "beto": sorted(glob("../data/runs/test*beto.csv")),
    "aya": sorted(glob("../data/runs/test*aya.csv")),
    "mixtral": sorted(glob("../data/runs/test*mixtral.csv")),
}

#assert len({len(value) for value in files.values()}) == 1

dfs = {k: pd.concat([pd.read_csv(f, index_col=0) for f in v]) for k, v in files.items()}
dfs["mixtral"]["pred_cot"] = dfs["mixtral"]["pred_cot"].apply(post_process_output)
{k: v.shape for k, v in dfs.items()}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


{'gpt-3.5': (5670, 24),
 'beto': (5670, 22),
 'aya': (5670, 25),
 'mixtral': (5670, 25)}

In [6]:
from rioplatense_hs.preprocessing import text_to_label, labels
from sklearn.metrics import classification_report

pred_labels = [f"PRED_{label}" for label in labels]
score_labels = [f"SCORE_{label}" for label in labels]

dfs["beto"][score_labels] = dfs["beto"][pred_labels]
dfs["beto"][pred_labels] = (dfs["beto"][score_labels] > 0.5).astype(int)


In [7]:
# Merge dfs['few-shot'] and dfs['beto']

columns_to_drop = [
    "title", "text", "context_tweet",
    # Remove gold labels
    "HATEFUL", "CALLS", "WOMEN", "LGBTI", "RACISM", "CLASS", "POLITICS", "DISABLED", "CRIMINAL", "APPEARANCE",
    # Remove predictions of dismissed labels
    "PRED_CALLS", "PRED_POLITICS", "PRED_DISABLED", "PRED_CRIMINAL", "PRED_APPEARANCE",
]

base_df = dfs["gpt-3.5"].copy()

base_df.drop([
    'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL',
    'PRED_POLITICS',
    'PRED_DISABLED', 'PRED_APPEARANCE', 'PRED_CRIMINAL', 'PRED_HATEFUL'
], axis=1, inplace=True)

print(base_df.columns)
df_join = base_df.join(dfs["beto"].drop(columns_to_drop, axis=1), rsuffix="_beto", lsuffix="_gpt")

mixtral = dfs["mixtral"].copy()

mixtral.drop([c for c in mixtral.columns if c in columns_to_drop], axis=1, inplace=True)
mixtral.rename(columns={f"PRED_{label}": f"PRED_{label}_mixtral" for label in all_labels}, inplace=True)
#mixtral.rename(columns={"pred_cot": "pred_cot_mixtral", "prompt": "prompt_mixtral"}, inplace=True)

df_join = df_join.join(mixtral, rsuffix="_mixtral", lsuffix="_gpt")

cols = [c for c in df_join.columns if c.isupper() and "SCORE" not in c]
#
df_join[cols] = df_join[cols].astype(bool)

sorted(df_join.columns)

Index(['title', 'text', 'context_tweet', 'HATEFUL', 'CALLS', 'WOMEN', 'LGBTI',
       'RACISM', 'CLASS', 'prompt', 'pred_cot', 'PRED_WOMEN', 'PRED_LGBTI',
       'PRED_RACISM', 'PRED_CLASS'],
      dtype='object')


['CALLS',
 'CLASS',
 'HATEFUL',
 'LGBTI',
 'PRED_CLASS_beto',
 'PRED_CLASS_gpt',
 'PRED_CLASS_mixtral',
 'PRED_HATEFUL',
 'PRED_LGBTI_beto',
 'PRED_LGBTI_gpt',
 'PRED_LGBTI_mixtral',
 'PRED_RACISM_beto',
 'PRED_RACISM_gpt',
 'PRED_RACISM_mixtral',
 'PRED_WOMEN_beto',
 'PRED_WOMEN_gpt',
 'PRED_WOMEN_mixtral',
 'RACISM',
 'SCORE_APPEARANCE',
 'SCORE_CLASS',
 'SCORE_CRIMINAL',
 'SCORE_DISABLED',
 'SCORE_LGBTI',
 'SCORE_POLITICS',
 'SCORE_RACISM',
 'SCORE_WOMEN',
 'WOMEN',
 'context_tweet',
 'error',
 'pred_cot_gpt',
 'pred_cot_mixtral',
 'prompt_gpt',
 'prompt_mixtral',
 'text',
 'title']

Ejemplos que falla BETO y acierta ChatGPT 

In [18]:
label = "RACISM"

beto_false_negatives_detected_by_chatgpt = df_join.loc[
    df_join[label] & df_join[f"PRED_{label}_gpt"] & ~df_join[f"PRED_{label}_beto"]
]

columns_to_show = ["context_tweet", "text", "pred_cot_gpt", "pred_cot_mixtral", f"SCORE_{label}"]

beto_false_negatives_detected_by_chatgpt[columns_to_show].sample(5)

,context_tweet,text,pred_cot_gpt,pred_cot_mixtral,SCORE_RACISM
id,,,,,
328692,"Ginés González García: ""El mundo nos elogia, cosa que habitualmente no sucede"" https://t.co/J0oqiW1Wgj",@usuario EL MUNDO ????? NO NOS REGISTRA SOMOS MAS ODIADOS QUE LOS MEXICANOS .,"El texto contiene un comentario despectivo hacia los mexicanos, comparándolos con los argentinos de manera negativa. Aunque no hace referencia directa a una característica protegida como mujer, lgbti, racismo o clase, provoca una actitud discriminatoria. La respuesta final es ""racismo"".",El texto no contiene discurso de odio. La respuesta final es 'nada'.,0.003089
378924,“Es aterrador”: la impunidad con que se mueve la armada pesquera de Xi Jinping en las costas de Ecuador https://t.co/Rc9CvGaZGu,@usuario Alguien le pone los puntos a los chinitos? O hacen lo que se les canta???,"El texto utiliza un término despectivo (""chinitos"") para referirse a los chinos, además de cuestionar su comportamiento y sugerir una jerarquía racial. La respuesta final es ""racismo"".","El texto se refiere a los chinos como ""chinitos"", un término despectivo. Además, sugiere que no tienen límites y que deben ser controlados. La respuesta final es ""racismo"".",0.081930
343308,China: Vuelven a vender murciélagos y carne de animales domésticos a pesar del Coronavirus #OtrasVezComenMurciélagos #YComenAnimalesDomésticos #AsíNo #QuéHorror #Coronavirus #China https://t.co/DTaFQaWzFH,@usuario Raza de mierda ☠️🦇,"El texto insulta a la ""raza"" china y utiliza un emoji de murciélago, relacionando a la pandemia de COVID-19 con una supuesta alimentación de murciélagos. La respuesta final es ""racismo"".","El texto muestra un desprecio hacia la raza china, utilizando un insulto. La respuesta final es ""racismo"".",0.064979
340832,Coronavirus: China registra 29 casos procedentes del exterior y solo 2 contagios a nivel local https://t.co/WaSDjJ1PV6,"@usuario Los únicos que le creen a los Chinos, son los Peronistas o alguna plaga similar.","El texto insulta a los chinos y a los peronistas, sugiriendo que solo ellos creen en la información proveniente de China. A pesar de no mencionar directamente una característica protegida, el insulto hacia los chinos se basa en prejuicios raciales. La respuesta final es ""racismo"".","El texto muestra un desprecio hacia China y su gobierno, y también hacia los peronistas. La respuesta final es ""racismo"".",0.024342
339477,El PJ elogió la actitud de China frente a la pandemia del coronavirus https://t.co/IYGVlArQuJ,@usuario Muchachos dejen de admirar tanto a los chinos porque esto del corona virus lo inventaron ellos para utilizarlo como arma bacteriologica y se le fue de las manos y esta a la vista las consecuencias.,"El texto contiene una teoría conspirativa y acusatoria hacia los chinos, sugiriendo que inventaron el coronavirus como un arma bacteriológica. Se basa en prejuicios y estereotipos hacia esta comunidad, por lo tanto, la respuesta final es ""racismo"".","El texto contiene una teoría conspirativa sobre el origen del coronavirus, acusando a China de ser responsable. La respuesta final es ""racismo"".",0.046612


In [20]:

not_detected_by_any = df_join.loc[
    df_join[label] & ~df_join[f"PRED_{label}_gpt"] & ~df_join[f"PRED_{label}_beto"]
]

not_detected_by_any[columns_to_show]

,context_tweet,text,pred_cot_gpt,pred_cot_mixtral,SCORE_RACISM
id,,,,,
347445,"Macabro hallazgo en China: 4.000 perros, gatos y otros animales muertos dentro de cajas en un depósito https://t.co/rUeDlGGdhj",@usuario Que hdp🤦,"El texto expresa desagrado hacia la acción de matar animales para consumo humano, pero no contiene discurso de odio hacia ninguna característica protegida. La respuesta final es ""nada"".",El texto no hace referencia a ninguna característica protegida. La respuesta final es 'nada'.,0.010623
340850,Coronavirus: China registra 29 casos procedentes del exterior y solo 2 contagios a nivel local https://t.co/WaSDjJ1PV6,@usuario Todo lo q digan los chinos es para dudar e investigar,"El texto sugiere que no se deben confiar en lo que digan los chinos, lo cual puede considerarse discriminatorio y desconfiado hacia ese grupo étnico. Sin embargo, no contiene un discurso de odio intenso o irracional. La respuesta final es ""nada"".",El texto muestra desconfianza hacia los chinos y su palabra. Esto puede ser interpretado como racismo. La respuesta final es 'racismo'.,0.026353
341186,"Coronavirus en Argentina: Tras los contactos entre Xi Jinping y Alberto Fernández, Aerolíneas Argentinas volará a China para traer una carga de insumos https://t.co/YLYJbzURE6",@usuario Los insumos vienen con coronavirus @usuario ?,"El texto no contiene discurso de odio ni menciona ninguna característica protegida. La respuesta final es ""nada"".",El texto no contiene discurso de odio. La pregunta es irónica y no está dirigida a ningún grupo protegido. La respuesta final es 'nada'.,0.000962
343293,China: Vuelven a vender murciélagos y carne de animales domésticos a pesar del Coronavirus #OtrasVezComenMurciélagos #YComenAnimalesDomésticos #AsíNo #QuéHorror #Coronavirus #China https://t.co/DTaFQaWzFH,@usuario No aprendieron nada espero que no saquen otra cepa peor al Coronavirus y el Estado no los sanciona?,"El texto no contiene discurso de odio. El mensaje simplemente expresa preocupación por la posibilidad de que en China continúen consumiendo este tipo de animales y que eso pueda dar lugar a una nueva cepa de virus. No se encuentran expresiones de rechazo irracional hacia un grupo específico. La respuesta es ""nada"".","El texto critica la decisión de China de volver a vender murciélagos y carne de animales domésticos. No se refiere a ninguna característica protegida, por lo que la respuesta final es 'nada'.",0.061544
335542,Coronavirus en la Argentina: El Presidente recibió a las organizaciones sociales y prometió fondos para la economía popular https://t.co/0P6jMH5BWB https://t.co/f85vCTGcLx,"@usuario Economía popular: trabajadores en negro, explotados y vagos planeros subsidiados hace 18 años.","El texto desprecia a las personas que se encuentran en la economía popular, llamándolos ""trabajadores en negro, explotados y vagos planeros subsidiados hace 18 años"". Esto muestra un desprecio hacia las clases bajas que requieren de la asistencia estatal, aludiendo a un motivo de clase. La respuesta final es ""clase"".","El texto desprecia a los trabajadores de la economía popular, llamándolos vagos y planeros. Esto muestra un desprecio hacia las clases bajas que requieren de la asistencia estatal. La respuesta final es ""clase"".",0.013998
339498,El PJ elogió la actitud de China frente a la pandemia del coronavirus https://t.co/IYGVlArQuJ,"@usuario Un fósil como @usuario preside un movimiento como el PJ, a contrapelo de la lógica y la verdad, elogiando a un régimen totalitario como el chino, q mintió en todo y además le agradecen por caramelos, cuando la comunidad internacional debería ser resarcida por el daño chino","El texto critica al Partido Justicialista por elogiar a China y lo califica como un ""fósil"", lo cual es despectivo pero no está dirigido a una característica protegida. Aunque menciona al régimen chino, no incurre en discurso de odio. La respuesta final es ""nada"".","El texto critica al PJ por elogiar a China, pero no con

In [16]:
# Show those which have one of these
# - Labeled as LGBTI
# - Detected by chatgpt as LGBTI
# - Detected by BETO as LGBTI
# - Detected by Mixtral

columns = [
    "context_tweet", "text", label,
    f"PRED_{label}_gpt", f"PRED_{label}_beto", f"PRED_{label}_mixtral",
    "pred_cot_gpt", "pred_cot_mixtral",
]
df_join.loc[
    df_join[label] | df_join[f"PRED_{label}_gpt"] | df_join[f"PRED_{label}_beto"] | df_join[f"PRED_{label}_mixtral"],
    columns
].to_csv("../data/tables/racism_analysis.csv")